## Regulation at the source? Comparing upstream and downstream climate policies

*An agent-based model to evaluate and compare the performance of upstream and downstream climate policies.*

Joël Foramitti, Ivan Savin, Jeroen C.J.M. van den Bergh

In [ ]:
## Libraries ##

import numpy as np
import pandas as pd
import csv

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from SALib.sample import saltelli
from SALib.analyze import sobol

In [ ]:
## Import Model ##

from UvsD_ABM_Parameters import *
from UvsD_ABM_Agents import *
from UvsD_ABM_Model import *

In [ ]:
## Settings ##
label = '_batch_combined_02_11'

open_economy = False 
do_multi_run = False
analyze_multi_run = True # Show evaluation criteria plot
analyze_sensitivity = True # Show sensitivity analysis plot
sensitivity_strength = 1000 # Number of runs: num_vars * 2 + 2
plot_sensitivity = True 
fs = 13 # Font size for plots

# Seperate simulation into smaller batches
batch_seperation = False
batch_total = 10
batch_current = 0 # start counting from 0!

In [ ]:
# Calculate relative values

def relative_measures(df):
    
    dfsc = df0['Scenario']
    df = df0.drop(['Scenario'], axis=1) # Remove scenario column
    
    st = str( len( df.index.unique() ) )
    
    for i in df.index.unique():
        df.loc[i] = df.loc[i].div( df.loc[i].abs().sum() ,axis=1)
        print('\rDone: ' + str(i+1) + ' out of ' + st, end='')
        
    df['Scenario'] = dfsc # Add scenario column back in
    
    return df

if analyze_multi_run == True:
    if open_economy: df0 = pd.read_csv('Outputs/results_UVSP_open'+label+'.csv')
    else: df0 = pd.read_csv('Outputs/results_UVSP'+label+'.csv')
    df0.rename(columns = {'Emission Costs': 'Emission Price'}, inplace = True)
    df0 = df0.set_index('Run')
    df = relative_measures(df0) 
    df.to_csv(r'Outputs/measures_relative_for_multi_UVSP.csv',index=False)

In [ ]:
# Plot bar chart

def multi_run_bar_chart(df,selected_criteria,ylim,figwidth=16,legend=False,label=''):
    
    sns.set()
    
    # Calculate relative values
    df0 = df.set_index('Scenario')
    x = len(selected_criteria)

    selected_measure_names = []
    for i in selected_criteria:
        selected_measure_names.append(measure_names2[i+2])
    
    # Run Statistics
    df1 = df0.iloc[:,selected_criteria]
    gp1 = df1.groupby(level=('Scenario'),sort=False)
    means = gp1.mean().T
    errors = gp1.std().T

    # Set up figure
    fig,ax = plt.subplots()
    
    # Plot Aggregate Values
    means.plot.bar(yerr=errors, 
                   capsize=3, 
                   figsize=(figwidth, 6),
                   ax=ax,
                   edgecolor='w',
                   color=['C0','C1','C2','C3','C1','C2','C3'])
    
    # Draw hatches
    bars = ax.patches
    hatches = [''] * x + [''] * x * 3 + ['//'] * x * 3 
    for bar, hatch in zip(bars, hatches):
        bar.set_hatch(hatch)
    
    plt.grid(axis='y', zorder=0)
    l=plt.legend(loc='upper left', bbox_to_anchor=(0, 1), ncol=4)
    if legend==False:l.remove() 

    plt.ylabel('Relative performance', fontsize = fs+1)
    plt.xticks(np.arange(x), selected_measure_names, rotation=0, fontsize=fs+1)
    plt.yticks(fontsize=fs)
    plt.savefig('Outputs/multi_run_bar_chart_'+label+'.pdf')
    plt.show() 
    
    return means
    
if analyze_multi_run == True:
    
    df = pd.read_csv('Outputs/measures_relative_for_multi_UVSP.csv') #_19_06
    means = multi_run_bar_chart(df,[1,2,3,5,11],ylim=0.6,legend=True,label='0') 
    multi_run_bar_chart(df,[0,15,12,13,8,9],ylim=0.6,legend=True,label='1') 

    if open_economy:
        multi_run_bar_chart(df,[x+15 for x in [0,1,2,3,5]],ylim=0.6,legend=True,label='0',open_economy=open_economy) 
        multi_run_bar_chart(df,[x+15 for x in [11,8,9,10]],ylim=0.6,legend=False,label='1',open_economy=open_economy) 
 

In [ ]:
# Sensitivity Analysis
# Add parameters to dataframe
# ['$λ$', '$Δθ$','$ΔA_0$,$ΔB_0$', 'Δμ', 'Δη','θ','σ','χ','μ','$N_s$ IS ETA!','γ','B_up_incr','for','cov','χup']
x = saltelli.sample(param_range, sensitivity_strength)
xx = [ i for i in x for y in range(len(scenarios))]
xxx = pd.DataFrame(xx,columns=param_range['names'])
df = pd.read_csv('Outputs/measures_relative_for_multi_UVSP.csv')
data = pd.concat([df,xxx],axis=1)

In [ ]:
data.head()

In [ ]:
d1 = data.query("Scenario != 'No Policy'")
scenarios = d1['Scenario'].unique()
parameters = ['$λ$', '$Δθ$','$ΔA_0$,$ΔB_0$', 'Δμ', 'Δη','θ','ϑ','$χ^M$','μ','η','γ','β','$t^*$','ϕ','$χ^S$']
measures = ['Technology adoption', 'Compositional change','Production decline','Sales price','Consumer Impact' ]

for p in parameters:
    
    for measure in measures:
        fig, ((ax1,ax2,ax3), (ax4,ax5,ax6)) = plt.subplots(nrows=2, ncols=3, figsize=(12, 7))
        axs = [ax1,ax2,ax3,ax4,ax5,ax6]

        for sc,ax in zip(scenarios,axs):
            data1 = data.query("Scenario == @sc")
            sns.regplot(x=p, y=measure, data=data1, ci=99, x_bins=10, fit_reg=False, label=measure, ax = ax) 
            ax.set_title(sc)

        plt.tight_layout()
        plt.show()
        
    #plt.savefig('Outputs/sensitivity_A.pdf')

In [ ]:
if analyze_sensitivity == True:   

    # Calculate Sobol Indices
    df = pd.read_csv('Outputs/results_UVSP'+label+'.csv') 
    #df = pd.read_csv('Outputs/measures_relative_for_multi_UVSP.csv') 
    
    Si = []
    
    sens_measures = list(df.columns) 
    sens_measures.remove('Scenario')
    
    scenarios = list(df['Scenario'].unique())
    
    i=1 # 15 measures, 7 scenarios = 105 Si
    for measure in sens_measures:
        
        m = []
        
        for scenario in scenarios:
            
            df1 = df.loc[df['Scenario'].isin([scenario])]
            
            y = np.array(df1[measure])
            
            m.append( sobol.analyze(param_range, y) )
            print("\rFinished calculating measure ",i,end="")
            i+=1
        
        Si.append(m)

    # Output: [ [m1 sc1, m1 sc2, ...] , [m2 sc1, ...] , ... ]
    Si = np.array(Si)
    
    np.save('Outputs/measure_names', sens_measures)
    np.save('Outputs/sensitivity_analysis'+label+'.npy', Si) 
    print("Done")

In [ ]:
# Plot Sensitivity Analysis - Single Order

if plot_sensitivity == True:
    
    fig, (ax1,ax2,ax3,ax4,ax5,ax6) = plt.subplots(nrows=6, ncols=1, figsize=(15, 20), sharex=True, sharey=True, gridspec_kw={'hspace':0})
    axis = (ax1,ax2,ax3,ax4,ax5,ax6)
    
    scenarios = ["No Policy","Downstream tax","Downstream permit market","Upstream Direct Regulation","Upstream tax","Upstream permit market"]
    measures = ['Emissions', 'Abatement Costs', 'Technology Adoption', 'Compositional Change', 'Product Sales', 'Consumer Impact','Leakage Effect']
    
    Si_all = np.load('Outputs/sensitivity_analysis'+label+'.npy', allow_pickle = True)
    measure_names_s = np.load('Outputs/measure_names.npy')
    colors = ["C0","C1","C2","C3","C4","C5"]
    
    for i in range(len(measure_names_s)): measure_names_s[i] = measure_names_s[i].replace('\r\n ', '')
    
    for i,ax in enumerate(axis): # Scenarios 
        
        for u,m in enumerate([x+1 for x in [1,2,3,5,11]]): # Measures
            
            Si = Si_all[m] # Select measure
            y = Si[i] # Select scenario
            c=1/6#(n+1)
            x = [c+j+c*u for j in list(range(len(y['S1'])))] 
            ax.errorbar( x , y['S1'], linestyle='', marker='.', label= measure_names_s[m]) 
    
            x = list(range(len(y['S1'])))
            y = list(np.arange(-0.1,0.8, 0.05))

            ax.set_yticks(y, minor = True)
            ax.set_ylabel(scenarios[i])
            
            ax.set_xticks(x, minor = True)
            ax.set_xticks([x+0.5 for x in x], minor = False)
            ax.grid(color='grey', linestyle='-',which='minor', axis='x')
            ax.grid(color='lightgrey', linestyle='-',which='major', axis='y')
            ax.set_xlim(0, 15)
            ax.set_ylim(-0.05, 0.8)
            
    ax6.set_xticklabels(param_range['names'], fontsize = fs)
    
    ax6.set_xlabel('Parameters', fontsize = fs+1)
    
    l=ax1.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
          fancybox=True, shadow=True, mode ="espand", ncol=5)
    
    if open_economy: plt.savefig('Outputs/sensitivity_chart_open.pdf')
    else: plt.savefig('Outputs/sensitivity_chart.pdf')
    
    plt.show()